# Setting the Random Seed
Author(s): Paul Miles | Date: June 19, 2018

For the purpose of testing the MCMC simulation on a particular problem, it may be useful to check whether the results are repeatable.  This can be accomplished by setting the seed for the random number generator used within `pymcmcstat`.  This tutorial outlines how to accomplish this, and demonstrates the repeatability of the results.

First we import the required packages, define our model/sum-of-squares functions, and define a data set.

In [9]:
import numpy as np
from pymcmcstat.MCMC import MCMC

# define test model function
def test_modelfun(xdata, theta):
    m = theta[0]
    b = theta[1]
    nrow, ncol = xdata.shape
    y = np.zeros([nrow,1])
    y[:,0] = m*xdata.reshape(nrow,) + b
    return y

def test_ssfun(theta, data):
    xdata = data.xdata[0]
    ydata = data.ydata[0]
    # eval model
    ymodel = test_modelfun(xdata, theta)
    # calc sos
    ss = sum((ymodel[:,0] - ydata[:,0])**2)
    return ss

# define data
nds = 100
x = np.linspace(2, 3, num=nds)
y = 2.*x + 3. + 0.1*np.random.standard_normal(x.shape)

### Initialize MCMC object and set seed
By default, no seed is specifield.  To specify a seed simply define a numeric value for the keywork argument `rngseed`.

In [14]:
# Initialize MCMC object
mcstat = MCMC(rngseed=1234)

### Setup and run rest of simulation
We set the
- data
- simulation options
- model settings
- model parameters

just like we would for any other simulation.

In [15]:
mcstat.data.add_data_set(x, y)
mcstat.simulation_options.define_simulation_options(nsimu = int(5.0e3), updatesigma = 1, method = 'dram')

# update model settings
mcstat.model_settings.define_model_settings(sos_function = test_ssfun)

mcstat.parameters.add_model_parameter(name = 'm', theta0 = 2., minimum = -10, maximum = np.inf, sample = 1)
mcstat.parameters.add_model_parameter(name = 'b', theta0 = -5., minimum = -10, maximum = 100, sample = 1)

# run mcmc
mcstat.run_simulation()


Sampling these parameters:
name         start [   min,    max] N(  mu, sigma^2)
m         :   2.00 [-10.00,    inf] N(0.00, inf)
b         :  -5.00 [-10.00, 100.00] N(0.00, inf)
 [-----------------100%-----------------] 5000 of 5000 complete in 1.1 sec

### Extract Results and Display Chainstats
In addition, we check the last row of the chain to see where the simulation ended.

In [16]:
# Extract results
results = mcstat.simulation_results.results

chain = results['chain']
s2chain = results['s2chain']
sschain = results['sschain']

names = results['names']

# define burnin
burnin = 2000
# display chain statistics
mcstat.chainstats(chain[burnin:,:], results)
print('chain[-1,:] = {}'.format(chain[-1,:]))


---------------------
name      :       mean        std     MC_err        tau     geweke
m         :     1.9876     0.0352     0.0047   107.1916     0.9782
b         :     3.0220     0.0891     0.0118   109.6284     0.9666
---------------------
chain[-1,:] = [1.95814523 3.1027293 ]


## Check Repeatability
To check the repeatability we simply create a new MCMC object with the same random seed and compare the result.

In [17]:
# Initialize MCMC object
mcstat = MCMC(rngseed=1234)

mcstat.data.add_data_set(x, y)
mcstat.simulation_options.define_simulation_options(nsimu = int(5.0e3), updatesigma = 1, method = 'dram')

# update model settings
mcstat.model_settings.define_model_settings(sos_function = test_ssfun)

mcstat.parameters.add_model_parameter(name = 'm', theta0 = 2., minimum = -10, maximum = np.inf, sample = 1)
mcstat.parameters.add_model_parameter(name = 'b', theta0 = -5., minimum = -10, maximum = 100, sample = 1)

# run mcmc
mcstat.run_simulation()

# Extract results
results = mcstat.simulation_results.results

chain = results['chain']
s2chain = results['s2chain']
sschain = results['sschain']

names = results['names']

# define burnin
burnin = 2000
# display chain statistics
mcstat.chainstats(chain[burnin:,:], results)
print('chain[-1,:] = {}'.format(chain[-1,:]))


Sampling these parameters:
name         start [   min,    max] N(  mu, sigma^2)
m         :   2.00 [-10.00,    inf] N(0.00, inf)
b         :  -5.00 [-10.00, 100.00] N(0.00, inf)
 [-----------------100%-----------------] 5000 of 5000 complete in 1.1 sec
---------------------
name      :       mean        std     MC_err        tau     geweke
m         :     1.9876     0.0352     0.0047   107.1916     0.9782
b         :     3.0220     0.0891     0.0118   109.6284     0.9666
---------------------
chain[-1,:] = [1.95814523 3.1027293 ]


It is clearly seen that the results are identical to the first simulation, so the random number process is repeatable.